In [ ]:
import os
import arcpy
import duckdb
import pyarrow as pa

from itables import init_notebook_mode
from itables import show

In [ ]:
init_notebook_mode(all_interactive=True)

In [ ]:
project = arcpy.mp.ArcGISProject("CURRENT")
duckdb_name = project.filePath.replace(".aprx", ".ddb")

In [ ]:
conn = duckdb.connect(duckdb_name, read_only=True)
_ = conn.execute("INSTALL spatial;LOAD spatial;")

In [ ]:
#show(
#    conn.sql("SELECT * FROM duckdb_settings()").df()
#)

In [ ]:
# sp_ref = project.activeMap.spatialReference
sp_ref = arcpy.SpatialReference(4230)
sp_ref

```sql
SELECT p.*
FROM pipeline_layer AS p
WHERE NOT EXISTS (
    SELECT 1
    FROM discovery_polygon_layer AS d
    WHERE ST_Intersects(p.geometry, d.geometry)
);
```

In [ ]:
pdf = conn.sql(f"""
WITH
L as (SELECT * FROM Pipelines),
R as (SELECT geom FROM Discoveries)
SELECT L.* EXCLUDE L.geom, ST_AsWKB(L.geom) SHAPE
FROM L LEFT JOIN R
ON ST_Intersects(L.geom, R.geom)
WHERE R.geom is NULL
""").df()

In [ ]:
# show(pdf.drop(["SHAPE"], axis="columns"))

In [ ]:
metadata = {"esri.encoding": "WKB", "esri.sr_wkt": sp_ref.exportToString()}
pa_shape = pa.field("SHAPE", pa.binary(), nullable=False, metadata=metadata)

In [ ]:
workspace = "memory"  # arcpy.env.scratchGDB
feature_class_name = "DeleteMe"

schema = pa.Schema.from_pandas(pdf)
shape_index = schema.get_field_index("SHAPE")
schema = schema.set(shape_index, pa_shape)
tab = pa.Table.from_pandas(pdf, schema=schema)
fc = os.path.join(workspace, feature_class_name)
arcpy.management.Delete(fc)
arcpy.management.CopyFeatures(tab, fc)

In [ ]:
conn.close()